## Что нужно сделать?
    (+) переделать получение описаний с запроса к api на поиск по данной html-странице
    (+) обработать все базы и выделить нужные вакансии, перенести в новую базу
    (+) выделить из каждой вакансии список требований
    (+) перенести сюда то, что скинул Саня
    (+) сделать окончательную удобную! базу (id, работодатель, дата публикации, город, требования) 
    
    
    (-) передалать выбор требований через regexp
    (-) сделать окончательный список встречающихся технологий
    (-) создать матрицу упоминаний технологий по каждой вакансии
    (-) подумать над тем, как лучше визуализировать результаты
    (-) визуализировать результаты
    (-) кластеризовать названия вакансий по их описаниям (опционально)

In [78]:
import sqlite3
import requests
import json
from bs4 import BeautifulSoup

In [77]:
pre_final = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\uni\\курсовая\\final.db'

conn = sqlite3.connect(pre_final)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
pre_final_db = cursor.fetchall()
conn.close()

In [76]:
final = 'dataset.db'

conn = sqlite3.connect(final)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancies')
final_db = cursor.fetchall()
conn.close()

### Создание новой "хорошей" базы данных
    1. используем hh.api для сбора полезных для инфографики характеристик

In [4]:
def collect_data(id_):
    req = requests.get('https://api.hh.ru/vacancies/' + id_).json()
    new_line = (
        id_,
        req['name'],
        req['area']['name'],
        req['employer']['name'],
        req['published_at'],
        req['description']
    )
    return new_line

In [5]:
#beautiful_db = []
#left = len(final_db)
#for job in final_db:
#    beautiful_db.append(collect_data(job[0]))
#    left -= 1
#    if left % 100 == 0:
#        print(f'осталось {left} вакансий.')        

In [6]:
#conn = sqlite3.connect('C:\\Users\\mag\\Desktop\\data_set.db')
#cursor = conn.cursor()
#cursor.execute('''CREATE TABLE vacancies (id text, name text, city text, 
#                                            employer text, publication_date text, description text)''')
#for job in beautiful_db:
#    try:
#        cursor.execute(f'''INSERT INTO vacancies VALUES {job}''')
#    except:
#        pass
#conn.commit()

### Формирование списка всех возможных технических навыков
    1. выкачка с википедии
    2. доп. анализ получившихся требований с целью обнаружения английский названий
    3. добавление аббревиатур

In [5]:
link = 'https://ru.wikipedia.org/wiki/Список_языков_программирования_по_категориям'
alphabet = 'абвгдеёжзийклмнопрстуфхцчщшьыъэюя'

In [6]:
soup = BeautifulSoup(requests.get(link).text, 'lxml')

In [7]:
base = [y for y in [x.text.lower() for x in soup.findAll('li')][20:-67] if set(y) & set(alphabet) == set()]

In [8]:
print(len(base))
base[-5:]

161


['lolcode', 'malbolge', 'piet', 'whitespace', 'emptyscript']

In [9]:
monosyllabic = [x for x in base if len(x.split()) == 1] # односложные названия
monosyllabic[:7]

['aspectc++', 'aspectj', 'caesarj', 'compose*', 'objectteams', 'basic', 'cg']

### Получение списка требований с помощью регулярных выражений
    то, что прислала Катя

In [28]:
# пример работы
list(filter(None, re.compile("<[^>]*>").split(re.sub('> +<', '', final_db[182][-1]))))

['Требования:',
 'Опыт работы в IT сфере;',
 'Понимание методологии ITSM;',
 'Знание интерфейса и опыт написание SQL скриптов;',
 'Навык разбора файлов в XML формате.',
 'Обязанности:',
 'Сопровождение АС Pega.',
 'Условия работы:',
 'Условия работы:',
 'Место работы: м. Кожуховская;',
 'График работы 5/2;',
 'Оформление согласно ТК РФ;',
 'Социальный пакет: ДМС, спортзал (фитнес, йога), спортивные и культурно-массовые мероприятия, возможность обучения за счет компании, льготные условия кредитования;',
 'Отличная профессиональная команда',
 'Годовая премия']

In [54]:
keywords1 = ['требования', 'желательно', 'необходимо', 'ожидаем', 'requirements', 'обязательно', 
            'приветствуется', 'знание', 'ожидания', 'компетенции', 'приветствуются', 
            'backend', 'технологии', 'требуется', 'если', 'ждём', 'важно',
            'навыки', 'будет', 'ищем', 'уровень', 'вас', 'дополнительно', 'желательны', 'техническое',
            'ждем', 'стек', 'ожиданем', 'умеете', 'приветствуем', 'навыки', 'нужно', 'стеке', 'потребуется', 
            'преимуществом', 'нужен', 'ожиданием', 'пригодится', 'expect', 'fullstack', 'качества', 
            'looking', 'хотелось', 'требовани', 'обязательн', 'must', 'skills', 'мастхэв', 'required',
            'ходим', 'обязателен', 'тебя', 'преимущества', 'знаниеумения', 'пожелания', 'стэк', 'компетенция', 
            'желательные', 'преимущество', 'хотим', 'знания', 'желание', 'хотели', 'have', 'применяется', 'технологий',
            'видим', 'обязательные', 'разработки']

keywords2 = ['плюсом', 'плюсами', 'бонусы', 'плюсы', 'плюс',]

punct = '«•»∙!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~'
table = str.maketrans({key: None for key in punct})

In [62]:
req_set = {}
errors = []
names = []

for job in final_db:
    #print(job[0])
    text = job[5]
    if '<strong>' in text:
        paragraphs = [x.split('</strong>') for x in text.split('<strong>') if '</strong>' in x]
        for x in paragraphs:
            if set(x[0].lower().translate(table).split()) & set(keywords) != set():
                names.append(x[0])
        #        req_set[job[0]] = list(filter(None, re.compile("<[^>]*>").split(re.sub('> +<', '', x[1]))))
        #       break
    else:
        errors.append(job[0])

In [71]:
names

['Требования:',
 'Требования: ',
 'Мы ждем тебя, чтобы строить счастливое будущее вместе!',
 'Требования: ',
 'Мы ждем тебя, чтобы строить счастливое будущее вместе!',
 'Требования:',
 'Навыки:',
 'Требования:',
 'Мы ждем тебя, чтобы строить счастливое будущее вместе!',
 'Технические компетенции:',
 'Дополнительные требования:',
 'Работая в данном проекте, разработчик получит возможность использовать в своей работе самые передовые технологии и фреймоворки BigData и стать одним из разработчиков такого фреймворка.',
 'Требования:',
 'Приветствуются знания / опыт:',
 'Требования:',
 'Мы ждем тебя, чтобы строить счастливое будущее вместе!',
 'Требования:',
 'ТРЕБОВАНИЯ К КАНДИДАТУ:',
 'Общие требования:',
 'Необходимые навыки и технологии:',
 'Будет плюсом:',
 'Мы ждем тебя, чтобы строить счастливое будущее вместе!',
 'Требования:',
 'Мы ищем ',
 'Технические компетенции',
 'Будет плюсом',
 'Уровень образования',
 'Бонусы',
 'Профессиональные компетенции',
 'Навыки',
 'Мы ожидаем от Вас:',

### Получение списка требований
    код, скопированный из предыдущего рабочего файла

In [11]:
import re

In [21]:
# удаление html-кода из текста
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return ' '.join(cleantext.split())

In [287]:
all_paragraphs = {}
errors = []
punct = '«•»∙!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~'
table = str.maketrans({key: None for key in punct})

for job in final_db:
    text = job[5]
    if '<strong>' in text:
        paragraphs = [x.split('</strong>') for x in text.split('<strong>') if '</strong>' in x]
        all_paragraphs[job[0]] = {x[0].translate(table).strip().lower(): x[1] for x in paragraphs}
    else:
        errors.append(job[0])

In [288]:
keywords = ['требования', 'желательно', 'плюсом', 'необходимо', 'ожидаем', 'requirements', 'обязательно', 
            'приветствуется', 'плюсами', 'знание', 'ожидания', 'компетенции', 'приветствуются', 
            'backend', 'технологии', 'требуется', 'если', 'бонусы', 'ждём', 'важно', 'плюсы', 'плюс', 
            'навыки', 'будет', 'ищем', 'уровень', 'вас', 'дополнительно', 'желательны', 'техническое',
            'ждем', 'стек', 'ожиданем', 'умеете', 'приветствуем', 'навыки', 'нужно', 'стеке', 'потребуется', 
            'преимуществом', 'нужен', 'ожиданием', 'пригодится', 'expect', 'fullstack', 'качества', 
            'looking', 'хотелось', 'требовани', 'обязательн', 'must', 'skills', 'мастхэв', 'required',
            'ходим', 'обязателен', 'тебя', 'преимущества', 'знаниеумения', 'пожелания', 'стэк', 'компетенция', 
            'желательные', 'преимущество', 'хотим', 'знания', 'желание', 'хотели', 'have', 'применяется', 'технологий',
            'видим', 'обязательные', 'разработки']

In [289]:
requirements = {}
requirements_l = []
for id_ in all_paragraphs: 
    for paragraph in all_paragraphs[id_]:
        word_list = paragraph.split()
        if set(word_list) & set(keywords) != set():              # а что если несколько заголовков подойдут?
            new_reqs = all_paragraphs[id_][paragraph].strip()
            if cleanhtml(new_reqs) != '':
                requirements[id_] = new_reqs
                requirements_l.append(new_reqs)
            else:
                errors.append(id_)

In [24]:
work_list = []
for i in requirements_l:
    work_list.append(i.split('>'))

In [25]:
new_work_list = []
for i in work_list:
    ap_list = []
    for j in i:
        ap_list.append(j.split('<'))
    new_work_list.append(ap_list)

In [26]:
flatten_list = []
for i in new_work_list:
    for j in i:
        for k in j:
            flatten_list.append(k)

In [27]:
trash_list = ['ul', 'li', 'p', 'b', '/ul', '/li', 
              '/p', '/b', 'em','/em','div','/div',
              'br','/br', 'ol', '/ol', '/span', 'u', '/u',
             ' ', '', 'br/', 'span', ':', 'br /']

In [28]:
requirements_final = []
for i in flatten_list:
    if i not in trash_list:
        requirements_final.append(i)

### Анализ наименований
    в конечном массиве каждая строка приведена к нижнему регистру, а и знаков препинания имеет только слэш, скобки и запятые.

In [162]:
new_punct = '«•»∙!"#$%&\'*-.:;<=>?@[\\]^_`{|}~'
new_table = str.maketrans({key: None for key in new_punct})
requirements = [x.translate(new_table).strip().lower() for x in requirements_final]

In [73]:
requirements[:5]

NameError: name 'requirements' is not defined

### Формирование окончательного списка технологий

In [175]:
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import collections

In [130]:
def prepare_text(text):
    return [gensim.utils.simple_preprocess(str(sentence), deacc=True) for sentence in text]

In [165]:
punct = ',)(«•»∙!"#$%&\'*-.:;<=>?@[\\]^_`{|}~'
table = str.maketrans({key: None for key in punct})
clean_requirements = [x.translate(table).strip().lower() for x in requirements]

In [166]:
techs = []
flatten = []
for sent in clean_requirements:
    line = []
    for token in sent.split():
        if set(token) & alph_en != set() and set(token) & alph_ru == set():
            line.append(token)
            flatten.append(token)
    if line != []:
        techs.append(line)

In [167]:
techs[:5]

[['ms', 'windows', 'ms', 'office'],
 ['macos/ios', 'linux'],
 ['windows'],
 ['web', 'html', 'html5', 'css', 'javascript', 'css'],
 ['angularjs', 'reactjs']]

In [168]:
bigram = gensim.models.phrases.Phrases(techs, min_count=4, threshold=10)
text_bigram = [bigram[sent] for sent in techs]

In [169]:
text_bigram[:5] # не очень чистый, но список технологий по предложениям

[['ms', 'windows', 'ms_office'],
 ['macos/ios', 'linux'],
 ['windows'],
 ['web', 'html', 'html5', 'css', 'javascript', 'css'],
 ['angularjs', 'reactjs']]

In [180]:
from_reqs_techs = [x[0] for x in sorted(dict(collections.Counter(flatten)).items(), key = lambda x: x[1], \
                                        reverse = True) if x[1] > 30]

In [178]:
from_reqs_techs = [x for x in from_reqs_techs if x not in stopwords.words('english')]

In [295]:
not_mine = ['a', 'and', 'd', 'e', 'etc', 'experience', 'in', 
            'intermediate', 'it', 'j', 'knowledge', 'of', 'e', 
            'j', 'self', 'the']

In [296]:
full_techs = set(list(set(from_reqs_techs + base).difference(not_mine)))

In [297]:
len(full_techs)

206

In [298]:
full_techs

{'5th',
 '8th',
 'abap/4',
 'ability',
 'action script',
 'active c#',
 'ada',
 'agile',
 'amp',
 'android',
 'angular',
 'ansible',
 'apache',
 'api',
 'aspectc++',
 'aspectj',
 'aspnet',
 'awk',
 'bash',
 'basic',
 'bcpl',
 'befunge',
 'brainfuck',
 'breeze',
 'byter',
 'c',
 'c#',
 'c++',
 'c++/cli',
 'c/c++',
 'caesarj',
 'cat',
 'cg',
 'chuck',
 'ci',
 'cilk',
 'clarion',
 'clean',
 'clipper',
 'coldfusion',
 'compose*',
 'confluence',
 'core',
 'corn',
 'cosy',
 'css',
 'curry',
 'data',
 'dataflex',
 'dbase',
 'delphi',
 'deque',
 'design',
 'development',
 'devops',
 'docker',
 'dylan',
 'eiffel',
 'elm',
 'emptyscript',
 'enchilada',
 'erlang',
 'euphoria',
 'excel',
 'f#',
 'factor',
 'false',
 'fbd',
 'forth',
 'foxpro',
 'framework',
 'freelang',
 'game maker language (gml)',
 'gentee',
 'git',
 'gnu bc',
 'golang',
 'gpss',
 'groovy',
 'haskell',
 'haxe',
 'hope',
 'html',
 'http',
 'ibm',
 'il',
 'intercal',
 'io',
 'ios',
 'is',
 'java',
 'java\njoin java\nx10',
 'javasc

### Что из списка требуется по каждой вакансии?

In [259]:
results = []
for job in final_db:
    line = {x:0 for x in full_techs}
    text = cleanhtml(job[5])
    for tech in full_techs:
        if tech in text:
            line[tech] = 1
    results.append(line)

In [235]:
import pandas as pd

In [249]:
len([x[3] for x in final_db])

1215

In [280]:
info_d = {
    'ids': [x[0] for x in final_db],
    'name': [x[1] for x in final_db],
    'city': [x[2] for x in final_db], 
    'employer': [x[3] for x in final_db],
    'publication date': [x[4] for x in final_db]
}

In [269]:
# лол, можно же как-то столбец добавлять а не вот это непонятно что
for tech in full_techs:
    line = [x[tech] for x in results]
    info_d.update({str(tech): line})

In [281]:
df = pd.DataFrame(info_d)

In [284]:
df.head()

ids                                               name    city  \
0  29654783  Инженер удаленной технической поддержки рабочи...  Самара   
1  29654641                               Frontend разработчик  Москва   
2  29654639                             JavaScript разработчик  Москва   
3  29654618                                    Ведущий инженер  Москва   
4  29668611                   Главный инженер по сопровождению  Москва   

   employer          publication date  
0  Сбербанк  2019-04-26T14:37:49+0300  
1  Сбербанк  2019-01-15T10:15:40+0300  
2  Сбербанк  2019-01-15T10:15:36+0300  
3  Сбербанк  2019-01-15T10:15:10+0300  
4  Сбербанк  2019-02-12T09:00:00+0300